In [5]:
import requests
from bs4 import BeautifulSoup
import time
import csv
#取得開始年
year_start = 2021
#取得終了年
year_end = 2022

for year in range(year_start, year_end):
    race_data_all = []
    #取得するデータのヘッダー情報を先に追加しておく
    race_data_all.append(['race_id','馬','騎手','馬番','走破時間','オッズ','通過順','着順','体重','体重変化','性','齢','斤量','上がり','人気','レース名','日付','開催','クラス','芝・ダート','距離','回り','馬場','天気','場id','場名'])
    List=[]
    #競馬場
    l=["01","02","03","04","05","06","07","08","09","10"]
    for w in range(len(l)):
        place = ""
        if l[w] == "01":
            place = "札幌"
        elif l[w] == "02":
            place = "函館"
        elif l[w] == "03":
            place = "福島"
        elif l[w] == "04":
            place = "新潟"
        elif l[w] == "05":
            place = "東京"
        elif l[w] == "06":
            place = "中山"
        elif l[w] == "07":
            place = "中京"
        elif l[w] == "08":
            place = "京都"
        elif l[w] == "09":
            place = "阪神"
        elif l[w] == "10":
            place = "小倉"

        #開催回数分ループ（6回）
        for z in range(7):
            continueCounter = 0  # 'continue'が実行された回数をカウントするためのカウンターを追加
            #開催日数分ループ（12日）
            for y in range(13):
                race_id = ''
                if y<9:
                    race_id = str(year)+l[w]+"0"+str(z+1)+"0"+str(y+1)
                    url1="https://db.netkeiba.com/race/"+race_id
                else:
                    race_id = str(year)+l[w]+"0"+str(z+1)+str(y+1)
                    url1="https://db.netkeiba.com/race/"+race_id
                #yの更新をbreakするためのカウンター
                yBreakCounter = 0
                #レース数分ループ（12R）
                for x in range(12):
                    if x<9:
                        url=url1+str("0")+str(x+1)
                        current_race_id = race_id+str("0")+str(x+1)
                    else:
                        url=url1+str(x+1)
                        current_race_id = race_id+str(x+1)
                    try:
                        r=requests.get(url)
                    #リクエストを投げすぎるとエラーになることがあるため
                    #失敗したら10秒待機してリトライする
                    except requests.exceptions.RequestException as e:
                        print(f"Error: {e}")
                        print("Retrying in 10 seconds...")
                        time.sleep(10)  # 10秒待機
                        r=requests.get(url)
                    #バグ対策でdecode
                    soup = BeautifulSoup(r.content.decode("euc-jp", "ignore"), "html.parser")
                    soup_span = soup.find_all("span")
                    #馬の数
                    allnum=(len(soup_span)-6)/3
                    #urlにデータがあるか判定
                    if allnum < 1:
                        yBreakCounter+=1
                        print('continue: ' + url)
                        continue
                    allnum=int(allnum)
                    race_data = []
                    for num in range(allnum):
                        #馬の情報
                        soup_txt_l=soup.find_all(class_="txt_l")
                        soup_txt_r=soup.find_all(class_="txt_r")
                        #走破時間
                        runtime=''
                        try:
                            runtime=soup_txt_r[2+5*num].contents[0]
                        except IndexError:
                            runtime = ''
                        soup_nowrap = soup.find_all("td",nowrap="nowrap",class_=None)
                        #通過順
                        pas = ''
                        try:
                            pas = str(soup_nowrap[3*num].contents[0])
                        except:
                            pas = ''
                        weight = 0
                        weight_dif = 0
                        #体重
                        var = soup_nowrap[3*num+1].contents[0]
                        try:
                            weight = int(var.split("(")[0])
                            weight_dif = int(var.split("(")[1][0:-1])
                        except ValueError:
                            weight = 0
                            weight_dif = 0
                        weight = weight
                        weight_dif = weight_dif

                        soup_tet_c = soup.find_all("td",nowrap="nowrap",class_="txt_c")
                        #上がり
                        last = ''
                        try:
                            last = soup_tet_c[6*num+3].contents[0].contents[0]
                        except IndexError:
                            last = ''
                        #人気
                        pop = ''
                        try:
                            pop = soup_span[3*num+10].contents[0]
                        except IndexError:
                            pop = ''
                        
                        #レースの情報
                        try:
                            var = soup_span[8]
                            sur=str(var).split("/")[0].split(">")[1][0]
                            rou=str(var).split("/")[0].split(">")[1][1]
                            dis=str(var).split("/")[0].split(">")[1].split("m")[0][-4:]
                            con=str(var).split("/")[2].split(":")[1][1]
                            wed=str(var).split("/")[1].split(":")[1][1]
                        except IndexError:
                            try:
                                var = soup_span[7]
                                sur=str(var).split("/")[0].split(">")[1][0]
                                rou=str(var).split("/")[0].split(">")[1][1]
                                dis=str(var).split("/")[0].split(">")[1].split("m")[0][-4:]
                                con=str(var).split("/")[2].split(":")[1][1]
                                wed=str(var).split("/")[1].split(":")[1][1]
                            except IndexError:
                                var = soup_span[6]
                                sur=str(var).split("/")[0].split(">")[1][0]
                                rou=str(var).split("/")[0].split(">")[1][1]
                                dis=str(var).split("/")[0].split(">")[1].split("m")[0][-4:]
                                con=str(var).split("/")[2].split(":")[1][1]
                                wed=str(var).split("/")[1].split(":")[1][1]
                        soup_smalltxt = soup.find_all("p",class_="smalltxt")
                        detail=str(soup_smalltxt).split(">")[1].split(" ")[1]
                        date=str(soup_smalltxt).split(">")[1].split(" ")[0]
                        clas=str(soup_smalltxt).split(">")[1].split(" ")[2].replace(u'\xa0', u' ').split(" ")[0]
                        title=str(soup.find_all("h1")[1]).split(">")[1].split("<")[0]

                        race_data = [
                            current_race_id,
                            soup_txt_l[4*num].contents[1].contents[0],#馬の名前
                            soup_txt_l[4*num+1].contents[1].contents[0],#騎手の名前
                            soup_txt_r[1+5*num].contents[0],#馬番
                            runtime,#走破時間
                            soup_txt_r[3+5*num].contents[0],#オッズ,
                            pas,#通過順
                            num+1,#着順
                            weight,#体重
                            weight_dif,#体重変化
                            soup_tet_c[6*num].contents[0][0],#性
                            soup_tet_c[6*num].contents[0][1],#齢
                            soup_tet_c[6*num+1].contents[0],#斤量
                            last,#上がり
                            pop,#人気,
                            title,#レース名
                            date,#日付
                            detail,
                            clas,#クラス
                            sur,#芝かダートか
                            dis,#距離
                            rou,#回り
                            con,#馬場状態
                            wed,#天気
                            w,#場
                            place]
                        race_data_all.append(race_data)
                    
                    print(detail+str(x+1)+"R")#進捗を表示
                    
                if yBreakCounter == 12:#12レース全部ない日が検出されたら、その開催中の最後の開催日と考える
                    break
    #1年毎に出力
    #出力先とファイル名は修正してください
    #年度ごとにファイルを分ける
    with open('data/'+str(year)+'.csv', 'w', newline='',encoding="SHIFT-JIS") as f:
        csv.writer(f).writerows(race_data_all)
    print("終了")

1回札幌1日目1R
1回札幌1日目2R
1回札幌1日目3R
1回札幌1日目4R
1回札幌1日目5R
1回札幌1日目6R
1回札幌1日目7R
1回札幌1日目8R
1回札幌1日目9R
1回札幌1日目10R
1回札幌1日目11R
1回札幌1日目12R
1回札幌2日目1R
1回札幌2日目2R
1回札幌2日目3R
1回札幌2日目4R
1回札幌2日目5R
1回札幌2日目6R
1回札幌2日目7R
1回札幌2日目8R
1回札幌2日目9R
1回札幌2日目10R
1回札幌2日目11R
1回札幌2日目12R
1回札幌3日目1R
1回札幌3日目2R
1回札幌3日目3R
1回札幌3日目4R
1回札幌3日目5R
1回札幌3日目6R
1回札幌3日目7R
1回札幌3日目8R
1回札幌3日目9R
1回札幌3日目10R
1回札幌3日目11R
1回札幌3日目12R
1回札幌4日目1R
1回札幌4日目2R
1回札幌4日目3R
1回札幌4日目4R
1回札幌4日目5R
1回札幌4日目6R
1回札幌4日目7R
1回札幌4日目8R
1回札幌4日目9R
1回札幌4日目10R
1回札幌4日目11R
1回札幌4日目12R
1回札幌5日目1R
1回札幌5日目2R
1回札幌5日目3R
1回札幌5日目4R
1回札幌5日目5R
1回札幌5日目6R
1回札幌5日目7R
1回札幌5日目8R
1回札幌5日目9R
1回札幌5日目10R
1回札幌5日目11R
1回札幌5日目12R
1回札幌6日目1R
1回札幌6日目2R
1回札幌6日目3R
1回札幌6日目4R
1回札幌6日目5R
1回札幌6日目6R
1回札幌6日目7R
1回札幌6日目8R
1回札幌6日目9R
1回札幌6日目10R
1回札幌6日目11R
1回札幌6日目12R
continue: https://db.netkeiba.com/race/202101010701
continue: https://db.netkeiba.com/race/202101010702
continue: https://db.netkeiba.com/race/202101010703
continue: https://db.netkeiba.com/race/202101010704
continue: https://db.netkeiba.com/race/202101010705
co

## 以下実験

### データの読み込み

In [14]:
import pandas as pd

# CSVファイルの読み込み（Shift_JISエンコーディングを指定）
df = pd.read_csv('./data/2016.csv', encoding='shift_jis')

# データの表示
print(df.shape)
df["race_id"].head()

(50076, 26)


0    201601010101
1    201601010101
2    201601010101
3    201601010101
4    201601010101
Name: race_id, dtype: int64

In [5]:
df.columns

Index(['race_id', '馬', '騎手', '馬番', '走破時間', 'オッズ', '通過順', '着順', '体重', '体重変化',
       '性', '齢', '斤量', '上がり', '人気', 'レース名', '日付', '開催', 'クラス', '芝・ダート', '距離',
       '回り', '馬場', '天気', '場id', '場名'],
      dtype='object')

In [15]:
df['race_id'].dtype


dtype('int64')

In [16]:
# 指定されたrace_idに一致する行をフィルタリング
filtered_df = df[df["race_id"] == 201601010101]

filtered_df

,race_id,馬,騎手,馬番,走破時間,オッズ,通過順,着順,体重,体重変化,...,日付,開催,クラス,芝・ダート,距離,回り,馬場,天気,場id,場名
0,201601010101,ニシノキララコ,井上敏樹,6,1:50.5,49.6,5-5-6-4,1,434,-4,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
1,201601010101,ユノディエール,四位洋文,2,1:50.8,29.6,8-7-7-6,2,438,-2,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
2,201601010101,マイネルヘミニス,丹内祐次,5,1:51.1,12.4,1-1-1-1,3,458,-2,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
3,201601010101,プラチナヴォイス,坂井瑠星,4,1:51.1,4.2,3-3-3-2,4,500,-4,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
4,201601010101,スピリットソウル,福永祐一,3,1:51.1,1.7,2-2-3-2,5,468,2,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
5,201601010101,ジャコマル,横山典弘,8,1:51.2,121.5,5-6-7-7,6,456,0,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
6,201601010101,クインズゴールド,吉田隼人,10,1:51.3,19.6,9-9-10-9,7,422,0,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
7,201601010101,アルマバテーム,勝浦正樹,9,1:52.5,117.8,7-7-7-10,8,488,-2,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
8,201601010101,ナルカミ,菱田裕二,7,1:53.3,8.7,9-9-2-5,9,470,8,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌
9,201601010101,プエルタデルソル,ティータ,1,1:54.8,14.8,3-3-5-7,10,462,-2,...,2016年7月30日,1回札幌1日目,2歳未勝利,芝,1800,右,良,曇,0,札幌


In [17]:
# 必要なカラム（"馬"と"オッズ"）を選択
result_df = filtered_df[["馬", "オッズ"]]

print(result_df)

          馬    オッズ
0   ニシノキララコ   49.6
1   ユノディエール   29.6
2  マイネルヘミニス   12.4
3  プラチナヴォイス    4.2
4  スピリットソウル    1.7
5     ジャコマル  121.5
6  クインズゴールド   19.6
7   アルマバテーム  117.8
8      ナルカミ    8.7
9  プエルタデルソル   14.8


### 以下三連複の計算方式

In [48]:
import pandas as pd

import itertools

# 仮定: result_df は各馬の名前とオッズを含むDataFrame
# 例: result_df = pd.DataFrame({'馬': ['馬A', '馬B', '馬C', ...], 'オッズ': [2.5, 5.0, 7.5, ...]})

# 支持率の計算
result_df['支持率'] = 1 / result_df['オッズ'] * 0.75

# 三連複の的中確率の計算
def calculate_trifecta_probability(df):
    oddsList = []
    # すべての異なる3頭の組み合わせを選択
    for horses in itertools.combinations(df['馬'], 3):
        A, B, C = horses
        A_support = df[df['馬'] == A]['支持率'].iloc[0]
        B_support = df[df['馬'] == B]['支持率'].iloc[0]
        C_support = df[df['馬'] == C]['支持率'].iloc[0]
        
        # 計算式に基づく確率の計算
        abc = A_support * B_support * C_support
        prob = abc * (
            1/ ((1 - A_support) * (1 - A_support - B_support)) + 
            1/ ((1 - B_support) * (1 - A_support - B_support)) +
            1/ ((1 - B_support) * (1 - B_support - C_support)) +
            1/ ((1 - C_support) * (1 - B_support - C_support)) +
            1/ ((1 - A_support) * (1 - A_support - C_support)) +
            1/ ((1 - C_support) * (1 - A_support - C_support)) 
        )
        odds = 0.75 / prob
        oddsList.append((horses, odds))

    return oddsList

# 三連複の的中確率の計算
trifecta_probabilities = calculate_trifecta_probability(result_df)

# 結果の表示（最初の5つの組み合わせのみ）
for combination in trifecta_probabilities[:100]:
    print(combination)

(('ニシノキララコ', 'ユノディエール', 'マイネルヘミニス'), 4856.75544404936)
(('ニシノキララコ', 'ユノディエール', 'プラチナヴォイス'), 1420.9897224465574)
(('ニシノキララコ', 'ユノディエール', 'スピリットソウル'), 353.1694482027515)
(('ニシノキララコ', 'ユノディエール', 'ジャコマル'), 50406.54137773357)
(('ニシノキララコ', 'ユノディエール', 'クインズゴールド'), 7864.815857576987)
(('ニシノキララコ', 'ユノディエール', 'アルマバテーム'), 48861.952401514754)
(('ニシノキララコ', 'ユノディエール', 'ナルカミ'), 3309.3983040114995)
(('ニシノキララコ', 'ユノディエール', 'プエルタデルソル'), 5859.7028687869115)
(('ニシノキララコ', 'マイネルヘミニス', 'プラチナヴォイス'), 572.7824681211109)
(('ニシノキララコ', 'マイネルヘミニス', 'スピリットソウル'), 141.51085307276378)
(('ニシノキララコ', 'マイネルヘミニス', 'ジャコマル'), 20332.396156043542)
(('ニシノキララコ', 'マイネルヘミニス', 'クインズゴールド'), 3172.2951646374313)
(('ニシノキララコ', 'マイネルヘミニス', 'アルマバテーム'), 19709.3553750688)
(('ニシノキララコ', 'マイネルヘミニス', 'ナルカミ'), 1334.6896939753274)
(('ニシノキララコ', 'マイネルヘミニス', 'プエルタデルソル'), 2363.4709204254414)
(('ニシノキララコ', 'プラチナヴォイス', 'スピリットソウル'), 39.75833138553448)
(('ニシノキララコ', 'プラチナヴォイス', 'ジャコマル'), 5950.318785672308)
(('ニシノキララコ', 'プラチナヴォイス', 'クインズゴールド'), 927.964830592

/var/folders/by/b0fpmcl53nz2m_3dxktpq4940000gn/T/ipykernel_59611/4082445226.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['支持率'] = 1 / result_df['オッズ'] * 0.75


In [49]:
import pandas as pd

# 三連複の的中確率の計算結果をDataFrameに変換
trifecta_df = pd.DataFrame(trifecta_probabilities, columns=['組み合わせ', 'オッズ'])

# CSVファイルに保存
trifecta_df.to_csv('trifecta_probabilities.csv', index=False)

# 確認のために最初の数行を表示
print(trifecta_df.head())

                          組み合わせ           オッズ
0  (ニシノキララコ, ユノディエール, マイネルヘミニス)   4856.755444
1  (ニシノキララコ, ユノディエール, プラチナヴォイス)   1420.989722
2  (ニシノキララコ, ユノディエール, スピリットソウル)    353.169448
3     (ニシノキララコ, ユノディエール, ジャコマル)  50406.541378
4  (ニシノキララコ, ユノディエール, クインズゴールド)   7864.815858


### 以下馬連の計算方式

In [55]:
import pandas as pd

import itertools

# 仮定: result_df は各馬の名前とオッズを含むDataFrame
# 例: result_df = pd.DataFrame({'馬': ['馬A', '馬B', '馬C', ...], 'オッズ': [2.5, 5.0, 7.5, ...]})

# 支持率の計算
result_df['支持率'] = 1 / result_df['オッズ'] * 0.775

# 三連複の的中確率の計算
def calculate_trifecta_probability(df):
    oddsList = []
    # すべての異なる3頭の組み合わせを選択
    for horses in itertools.combinations(df['馬'], 2):
        A, B = horses
        A_support = df[df['馬'] == A]['支持率'].iloc[0]
        B_support = df[df['馬'] == B]['支持率'].iloc[0]
        
        # 計算式に基づく確率の計算
        ab = A_support * B_support 
        prob = ab * (
            1/ (1 - A_support)  + 
            1/ (1 - B_support) 
        )
        odds = 0.75 / prob
        oddsList.append((horses, odds))

    return oddsList

# 三連複の的中確率の計算
boublefecta_probabilities = calculate_trifecta_probability(result_df)

# # 結果の表示（最初の5つの組み合わせのみ）
# for combination in boublefecta_probabilities[:100]:
#     print(combination)

/var/folders/by/b0fpmcl53nz2m_3dxktpq4940000gn/T/ipykernel_59611/3439128745.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['支持率'] = 1 / result_df['オッズ'] * 0.775


In [56]:
import pandas as pd

# 三連複の的中確率の計算結果をDataFrameに変換
doublefecta_df = pd.DataFrame(boublefecta_probabilities, columns=['組み合わせ', 'オッズ'])

# CSVファイルに保存
doublefecta_df.to_csv('boublefecta_probabilities.csv', index=False)

# 確認のために最初の数行を表示
print(doublefecta_df.head())

                 組み合わせ          オッズ
0   (ニシノキララコ, ユノディエール)   897.457783
1  (ニシノキララコ, マイネルヘミニス)   368.780488
2  (ニシノキララコ, プラチナヴォイス)   116.017656
3  (ニシノキララコ, スピリットソウル)    36.895932
4     (ニシノキララコ, ジャコマル)  3721.104168
